<a href="https://colab.research.google.com/github/BiancaStadl/ProjektarbeitML/blob/main/LSTM_limited_vocab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Quelle für Twitter-Daten: https://github.com/uds-lsv/GermEval-2018-Data/

Michael Wiegand, Melanie Siegel, and Josef Ruppenhofer: "Overview of the GermEval 2018 Shared Task on the Identification of Offensive Language", in Proceedings of the GermEval, 2018, Vienna, Austria.

In [ ]:
#import matplotlib.pyplot as plt -> für evtl Visualisierungen
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
from keras import losses
from tensorflow import keras 
from keras import optimizers 
from keras import metrics 

#!pip install Tokenizer
from keras.preprocessing.text import Tokenizer
#!pip install pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
#from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


Einige Parameter für das Netz setzen.. https://becominghuman.ai/creating-your-own-neural-network-using-tensorflow-fa8ca7cc4d0e 

In [ ]:
training_epochs = 10
batch_size = 32
max_length = 60
max_vocab_size=6000

In [ ]:
url = "https://github.com/uds-lsv/GermEval-2018-Data/archive/master.zip"

dataset = tf.keras.utils.get_file("GermEval-2018-Data-master.zip", url, 
                                   extract=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'GermEval-2018-Data-master')


9068544/Unknown - 1s 0us/step

In [ ]:
os.listdir(dataset_dir)

['LICENSE.md',
 'results.pdf',
 'germeval2018.test.txt',
 'README.md',
 'submissions.tgz',
 'germeval2018.training.txt',
 'survey.pdf',
 'guidelines-iggsa-shared.pdf',
 'evaluationScriptGermeval2018.pl']

In [ ]:
training_file = os.path.join(dataset_dir, 'germeval2018.training.txt')
#with open(training_file) as f:
 # print(f.read())

#print()

testing_file = os.path.join(dataset_dir, 'germeval2018.test.txt')


In [ ]:
def remove_clutter(string):
   string = re.sub("@[^\s]+"," ",string)
   string = re.sub("#[^\s]+"," ", string)
   string = re.sub("\u00a9"," ", string)
   string = re.sub("\u00ae"," ", string)
   string = re.sub("[\u2000-\u3300]"," ", string)
   string = re.sub("\ud83c[\ud000-\udfff]"," ", string)
   string = re.sub("\ud83d[\ud000-\udfff]"," ", string)
   string = re.sub("\ud83e[\ud000-\udfff]"," ", string)
   string = re.sub("😜", " ",string)
   string = re.sub("🍫", " ",string)
   string = re.sub("😁", " ",string)
   string = re.sub("🐖", " ",string)
   string = re.sub("😡", " ",string)
   string = re.sub("😇", " ",string)
   string = re.sub("😬", " ",string)
   string = re.sub("😃", " ",string)
   string = re.sub("😂", " ",string)
   string = re.sub("💙", " ",string)  
   string = re.sub("😛", " ",string)
   string = re.sub("🙏", " ",string)
   string = re.sub("👍", " ",string)
   string = re.sub("🖕", " ",string)
   string = re.sub("😉", " ",string)
   string = re.sub("💩", " ",string)
   string = re.sub("🤢", " ",string)
   string = re.sub("👏", " ",string)
   string = re.sub("😨", " ",string)
   string = re.sub("🤣", " ",string)
   string = re.sub("🤡", " ",string)
   string = re.sub("😈", " ",string)
   string = re.sub("💃🏽", " ",string)
   string = re.sub("👹", " ",string)
   string = re.sub("🤘", " ",string)
   string = re.sub("😱", " ",string)
   string = re.sub("🤔", " ",string) 
   string = re.sub("🌈", " ",string) 
   string = re.sub("💕", " ",string) 
   string = re.sub("👩‍❤️‍👩", " ",string) 
   string = re.sub("😍", " ",string) 
   string = re.sub("👆", " ",string) 
   string = re.sub("😖", " ",string) 
   string = re.sub("👇", " ",string) 
   string = re.sub("🔥", " ",string) 
   string = re.sub("😘", " ",string) 
   string = re.sub("🎉", " ",string) 
   string = re.sub("🤬", " ",string) 
   string = re.sub("👊", " ",string)
   string = re.sub("🇩🇪", " ",string)  
   
   string = re.sub("OTHER|OFFENSE|ABUSE|INSULT"," ",string)
   return string

In [ ]:
statementsForTraining = []
sentimentsForTraining = []

fileToRead = open(training_file, 'r')

while True:
  #next line in file
  line = fileToRead.readline()

  if line == "":
   break

  findSentiment = re.search("OTHER|OFFENSE",line)

  line = remove_clutter(line)
      

  statementsForTraining.append(line)

   #sentimentsForTraining.append(findSentiment.group(0))

  if findSentiment.group(0) == "OTHER":  
    sentimentsForTraining.append(0)
  else:
    sentimentsForTraining.append(1)

  if not line:
    break

 #print("{}: {}".format(count,line.strip()))
  
 # print(sentiment.group(0))
 
fileToRead.close()

training_sentences = statementsForTraining
training_labels = sentimentsForTraining

#print(training_sentences[0:100])
#print(training_labels[9])  

In [ ]:

statementsForTesting = []
sentimentsForTesting = []

fileToRead = open(testing_file, 'r')

while True:
 
  line = fileToRead.readline()

  if line == "":
   break

  sent = re.search("OTHER|OFFENSE",line)

  line = remove_clutter(line)


  statementsForTesting.append(line)
  #print(len(line))
  #sentimentsForTesting.append(sent.group(0))

  if sent.group(0) == "OTHER": 
    sentimentsForTesting.append(0)
  else:
    sentimentsForTesting.append(1)

  if not line:
    break


fileToRead.close()


testing_sentences = statementsForTesting
testing_labels = sentimentsForTesting
#print(len(testing_sentences))
#print(testing_sentences)   
#print(statementsForTesting)

Ansehen (wegen num_words) https://github.com/keras-team/keras/issues/8092 

In [ ]:
tokenizer = Tokenizer(num_words=6000, oov_token="OOV")
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

validation_size = 500

training_sequences = tokenizer.texts_to_sequences(training_sentences)
padded_training = pad_sequences(training_sequences, maxlen=max_length, padding='post')

#print(len(padded_training))

validation_sequences = padded_training[0:validation_size]
validation_labels = training_labels[0:validation_size]

padded_training = padded_training[validation_size:]
training_labels = training_labels[validation_size:]

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
padded_testing = pad_sequences(testing_sequences, maxlen=max_length, padding='post')

#print(validation_sequences[499])
#print(padded_training[0])
#print(len(validation_labels))
#print(len(training_labels))


nppadded_training = np.array(padded_training)
nptraining_labels = np.array(training_labels)

nppadded_validation = np.array(validation_sequences)
npvalidation_labels = np.array(validation_labels)

nppadded_testing = np.array(padded_testing)
nptesting_labels = np.array(testing_labels)




In [ ]:
#print(statementsForTraining[2])

#print(nppadded_training[4])
#print(nppadded_training.shape)

print(nptraining_labels[4])
#print(nppadded_testing.shape)

#print(word_index) 

1


In [ ]:
from google.colab import drive
drive.mount("/content/drive")
os.listdir("/content/drive/MyDrive/Colab Notebooks")

Mounted at /content/drive


['germeval_training.txt',
 'glove.twitter.27B.50d.txt',
 'glove.twitter.27B.200d.txt',
 'glove.840B.300d.txt',
 'tensorboard.gdoc',
 'keras.gdoc',
 'CNN embedded.ipynb',
 'CNN embedded_limited vocab.ipynb',
 'LSTM.ipynb',
 'LSTM_limited_vocab.ipynb']

In [ ]:
#Größe Vokabel -> wordindex + 2 (weil padding + OOV) 
hits = 0
misses = 0

#vocabulary_size = len(word_index)+2
vocabulary_size = 6000

# dann erstell ich ein Wörterbuch mit Namen "embedding_vector", dort sind dann
#die keys drinnen, die in glove-Datei drinnen sind mit dem entsprechenden Key

embedding_index_glove = {}
f = open('/content/drive/MyDrive/Colab Notebooks/glove.twitter.27B.200d.txt')
for line in f:
  value = line.split()
  word = value[0]
  coef = np.asarray(value[1:],dtype='float32')
  embedding_index_glove[word] = coef

print("%d gefunden: "% len(embedding_index_glove))

#Dann noch eine Embedding-Matrix erstellen
#zweiter Wert = Embedding-Dimension der Datei, in dem Fall 200

glove_matrix = np.zeros((vocabulary_size,200))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size-1:
      break
    else:
      embedding_value = embedding_index_glove.get(word)
      if embedding_value is not None:
        glove_matrix[index] = embedding_value
        hits+=1
      else:
        misses+=1

print("hits %d and %d misses"%(hits,misses))

1193514 gefunden: 
hits 4103 and 1896 misses


In [ ]:
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, LSTM

output-dimension -> Werte variieren.. Zuerst 16, da war die Accuracy nur etwas über 60, bei LSTM 10.
Dann Output-dim auf 120 und LSTM auf 10 -> Accuracy fast bei 90%

dann LSTM auf 265 und dim auf 120 -> knapp über 60

In [ ]:
import keras.backend as K

def metrics_recall(data_true, data_pred):
    data_true = K.ones_like(data_true)
    true_positives = K.sum(K.round(K.clip(data_true*data_pred,0,1)))
    possible_positives = K.sum(K.round(K.clip(data_true,0,1)))

    recall = true_positives / (possible_positives+K.epsilon())
    return recall


def metrics_precision(data_true, data_pred):
    data_true = K.ones_like(data_true)
    true_positives = K.sum(K.round(K.clip(data_true*data_pred,0,1)))

    positives_predicted = K.sum(K.round(K.clip(data_pred,0,1)))
    precision = true_positives / (positives_predicted+K.epsilon())
    return precision


def metrics_f1(data_true, data_pred):
    precision_data = metrics_precision(data_true, data_pred)
    recall_data = metrics_recall(data_true, data_pred)
    return 2*(precision_data*recall_data)/(precision_data+recall_data+K.epsilon())

In [ ]:
from tensorflow.keras.layers import Embedding
print(len(word_index))
model_LSTM = tf.keras.Sequential()
#model_LSTM.add(tf.keras.layers.Embedding(input_dim=15132, output_dim=100, input_length=max_length))
model_LSTM.add(tf.keras.layers.Embedding(vocabulary_size, output_dim=200, input_length=60, embeddings_initializer = keras.initializers.Constant(glove_matrix), trainable= False))
model_LSTM.add(tf.keras.layers.LSTM(95, activation="relu", use_bias=True, return_sequences=True))
model_LSTM.add(tf.keras.layers.LSTM(95))
model_LSTM.add(tf.keras.layers.Dropout(0.5))
model_LSTM.add(tf.keras.layers.Dense(64,activation='relu'))
model_LSTM.add(tf.keras.layers.Dense(1))

model_LSTM.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',metrics_recall,metrics_precision,metrics_f1])
print(model_LSTM.summary())

15130
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 60, 200)           1200000   
_________________________________________________________________
lstm (LSTM)                  (None, 60, 95)            112480    
_________________________________________________________________
lstm_1 (LSTM)                (None, 95)                72580     
_________________________________________________________________
dropout (Dropout)            (None, 95)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                6144      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 1,391,269
Trainable params: 191,269
Non-trainable params: 1,200,000
__________________________________

In [ ]:
model_LSTM.layers[1].get_weights()[0].shape

In [ ]:
%load_ext tensorboard

In [ ]:
logs_base_dir = "./logs"
callbackForTB = tf.keras.callbacks.TensorBoard(logs_base_dir)

In [ ]:
model_LSTM.fit(nppadded_training, nptraining_labels, batch_size=128, epochs=10, validation_data=(nppadded_testing, nptesting_labels), callbacks=[callbackForTB])

Epoch 1/10
36/36 [==============================] - 17s 376ms/step - loss: 1.5469 - accuracy: 0.6476 - metrics_recall: 0.0584 - metrics_precision: 0.4933 - metrics_f1: 0.1020 - val_loss: 0.6455 - val_accuracy: 0.6597 - val_metrics_recall: 0.0000e+00 - val_metrics_precision: 0.0000e+00 - val_metrics_f1: 0.0000e+00
Epoch 2/10
36/36 [==============================] - 12s 347ms/step - loss: 0.6552 - accuracy: 0.6379 - metrics_recall: 0.0809 - metrics_precision: 0.9969 - metrics_f1: 0.1455 - val_loss: 0.6645 - val_accuracy: 0.6597 - val_metrics_recall: 0.0000e+00 - val_metrics_precision: 0.0000e+00 - val_metrics_f1: 0.0000e+00
Epoch 3/10
36/36 [==============================] - 12s 345ms/step - loss: 0.6575 - accuracy: 0.6460 - metrics_recall: 0.0271 - metrics_precision: 0.8557 - metrics_f1: 0.0507 - val_loss: 0.6519 - val_accuracy: 0.6597 - val_metrics_recall: 0.0000e+00 - val_metrics_precision: 0.0000e+00 - val_metrics_f1: 0.0000e+00
Epoch 4/10
36/36 [==============================] - 12s

In [ ]:
%tensorboard --logdir {logs_base_dir}

In [ ]:
from tensorboard import notebook
notebook.list()

In [ ]:
notebook.display(port=6006, height=1000) 
#error..

In [ ]:
#!pip install tensorboardcolab
#from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback

#tbc=TensorBoardColab()

In [ ]:
results = model_LSTM.evaluate(nppadded_testing, nptesting_labels, batch_size=batch_size)
print("test loss, test acc:",results)

In [ ]:
model_LSTM.predict(nppadded_testing[10:16])

In [ ]:
print(testing_labels[10:16])
#print(statementsForTesting[4:7])